In [1]:
# Section must be included at the beginning of each new notebook. Remember to change the app name. 
# If you're using VirtualBox, change the below to '/home/user/spark-2.1.1-bin-hadoop2.7'
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [2]:
# Let's read in the data. If you open the dataset, you'll find that each column has a header. We specify that by stating that header=True.
# To make our lives easier, we can also use 'inferSchema' when importing CSVs. This automatically detects data types.
# If you would like to manually change data types, refer to this article: https://medium.com/@mrpowers/adding-structtype-columns-to-spark-dataframes-b44125409803
df = spark.read.csv('absenteeism3.csv',header=True,inferSchema=True)

In [3]:
# The show method allows you visualise DataFrames in a tabular format. 
df.show()

+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| ID|Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time|Age|Work load Average/day |Hit target|Disciplinary failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Body mass index|Absenteeism time in hours|Height|
+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| 11|                26|               7|              3|      1|                   289|                        

In [4]:
# Print schema allows us to visualise the data structure at a high level. 
df.printSchema()

# We can also use head to print a specific amount of rows, so we can get a better understanding of the data points. 
# Note that we have to specify 'print' depending on the method we're using. Otherwise it may not show up!
print(df.head(1))

root
 |-- ID: integer (nullable = true)
 |-- Reason for absence: integer (nullable = true)
 |-- Month of absence: integer (nullable = true)
 |-- Day of the week: integer (nullable = true)
 |-- Seasons: integer (nullable = true)
 |-- Transportation expense: integer (nullable = true)
 |-- Distance from Residence to Work: integer (nullable = true)
 |-- Service time: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Work load Average/day : string (nullable = true)
 |-- Hit target: integer (nullable = true)
 |-- Disciplinary failure: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- Son: integer (nullable = true)
 |-- Social drinker: integer (nullable = true)
 |-- Social smoker: integer (nullable = true)
 |-- Pet: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Body mass index: integer (nullable = true)
 |-- Absenteeism time in hours: integer (nullable = true)
 |-- Height: double (nullable = true)

[Row(ID=11, Reason for absence=

In [5]:
# We can use the describe method get some general statistics on our data too. 
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+----------------------+-------------------------------+------------------+-----------------+----------------------+-----------------+--------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------------+--------------------+
|summary|                ID|Reason for absence|  Month of absence|   Day of the week|           Seasons|Transportation expense|Distance from Residence to Work|      Service time|              Age|Work load Average/day |       Hit target|Disciplinary failure|         Education|               Son|     Social drinker|      Social smoker|               Pet|            Weight|   Body mass index|Absenteeism time in hours|              Height|
+-------+------------------+------------------+------------------+------------------+------------------+--------------

In [6]:
# Let's select the columns that are integers, and use the describe method again.
# We see that the average age is 41. The average bank account balance is $1,074. 
# And they spoke to call centre reps for approx. 931 seconds on average. 
df.select('ID', 'Reason for absence', 'age','Education', 'Son', 'Social drinker', 'Social smoker', 'Pet', 'Weight', 'Body mass index', 'Absenteeism time in hours', 'Height').describe().show()

+-------+------------------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------------+--------------------+
|summary|                ID|Reason for absence|              age|         Education|               Son|     Social drinker|      Social smoker|               Pet|            Weight|   Body mass index|Absenteeism time in hours|              Height|
+-------+------------------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------------+--------------------+
|  count|               740|               740|              740|               740|               740|                740|                740|               740|               740|               740|                      740|                 740|
|   mean

In [7]:
# Let's select the balance column and assign it to a variable. 
df= df.select('ID', 'Reason for absence', 'age','Education', 'Son', 'Social drinker', 'Social smoker', 'Pet', 'Weight', 'Body mass index', 'Absenteeism time in hours', 'Height')

# We can then use the show method on that variable.
df.show()

+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| ID|Reason for absence|age|Education|Son|Social drinker|Social smoker|Pet|Weight|Body mass index|Absenteeism time in hours|Height|
+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| 11|                26| 33|        1|  2|             1|            0|  1|    90|             30|                        4|  1.72|
| 36|                 0| 50|        1|  1|             1|            0|  0|    98|             31|                        0|  1.78|
|  3|                23| 38|        1|  0|             1|            0|  0|    89|             31|                        2|   1.7|
|  7|                 7| 39|        1|  2|             1|            1|  0|    68|             24|                        4|  1.68|
| 11|                23| 33|        1|  2|             1|            0|  1| 

In [8]:
# Let's try out some additional DataFrame methods.
# How would we identify individuals with a balance above $5,000? Using filter! 
df.filter("ID=1").show()

# We can also use more advanced filters. For example, let's see the jobs of people with over $2,500 in their bank account.
df.filter("ID=1").select('Absenteeism time in hours','Reason for absence').show()

+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| ID|Reason for absence|age|Education|Son|Social drinker|Social smoker|Pet|Weight|Body mass index|Absenteeism time in hours|Height|
+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
|  1|                22| 37|        3|  1|             0|            0|  1|    88|             29|                        8|  1.72|
|  1|                23| 37|        3|  1|             0|            0|  1|    88|             29|                        4|  1.72|
|  1|                26| 37|        3|  1|             0|            0|  1|    88|             29|                        8|  1.72|
|  1|                 7| 37|        3|  1|             0|            0|  1|    88|             29|                        3|  1.72|
|  1|                13| 37|        3|  1|             0|            0|  1| 

In [9]:
# Let's try out some additional DataFrame methods.
# How would we identify individuals with a balance above $5,000? Using filter! 
df.filter("ID=18").show()

# We can also use more advanced filters. For example, let's see the jobs of people with over $2,500 in their bank account.
df.filter("ID=18").select('Absenteeism time in hours','Reason for absence').show()

+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| ID|Reason for absence|age|Education|Son|Social drinker|Social smoker|Pet|Weight|Body mass index|Absenteeism time in hours|Height|
+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| 18|                10| 28|        2|  0|             0|            0|  0|    84|             25|                        8|  1.82|
| 18|                18| 28|        2|  0|             0|            0|  0|    84|             25|                        8|  1.82|
| 18|                23| 28|        2|  0|             0|            0|  0|    84|             25|                        1|  1.82|
| 18|                 0| 28|        2|  0|             0|            0|  0|    84|             25|                        0|  1.82|
| 18|                26| 28|        2|  0|             0|            0|  0| 

In [10]:
# To simplify things, let's split this into two steps. First, let's create a variable then order by age.
# Careful when using show()! Otherwise the variable type will change and you won't be able to order it. 
group_job_df = df.groupBy('ID').mean()

# Note that we have to use 'avg(age)' instead of age. Why? Because when you use mean(), it changes the feature's name (as you can see below).
print("Sorted by Absenteeism time in hours")
group_job_df.orderBy('avg(Absenteeism time in hours)').show()

# Let's see what this looks like in one line.
print("Sorted by Body mass index")
df.groupBy('ID').mean().orderBy('avg(Body mass index)').show()


Sorted by Absenteeism time in hours
+---+-------+-----------------------+--------+--------------+--------+-------------------+------------------+--------+-----------+--------------------+------------------------------+------------------+
| ID|avg(ID)|avg(Reason for absence)|avg(age)|avg(Education)|avg(Son)|avg(Social drinker)|avg(Social smoker)|avg(Pet)|avg(Weight)|avg(Body mass index)|avg(Absenteeism time in hours)|       avg(Height)|
+---+-------+-----------------------+--------+--------------+--------+-------------------+------------------+--------+-----------+--------------------+------------------------------+------------------+
|  8|    8.0|                    0.0|    39.0|           1.0|     2.0|                1.0|               0.0|     2.0|      100.0|                35.0|                           0.0|               1.7|
| 35|   35.0|                    0.0|    53.0|           1.0|     1.0|                0.0|               0.0|     1.0|       77.0|                25.0|     

In [11]:
# Let's select the columns that are integers, and use the describe method again.
# We see that the average age is 41. The average bank account balance is $1,074. 
# And they spoke to call centre reps for approx. 931 seconds on average. 
df.select('ID', 'Reason for absence', 'age','Education', 'Son', 'Social drinker', 'Social smoker', 'Pet', 'Weight', 'Body mass index', 'Absenteeism time in hours', 'Height').describe().show()

+-------+------------------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------------+--------------------+
|summary|                ID|Reason for absence|              age|         Education|               Son|     Social drinker|      Social smoker|               Pet|            Weight|   Body mass index|Absenteeism time in hours|              Height|
+-------+------------------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------------+--------------------+
|  count|               740|               740|              740|               740|               740|                740|                740|               740|               740|               740|                      740|                 740|
|   mean

In [12]:
#clean data

In [13]:
#import findspark
#findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
#import pyspark
#from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('missing').getOrCreate()

In [14]:
df= df.select('ID', 'Reason for absence', 'age','Education', 'Son', 'Social drinker', 'Social smoker', 'Pet', 'Weight', 'Body mass index', 'Absenteeism time in hours', 'Height')

# Let's visually inspect the data. You may notice some nulls in the marital and balance column. 
df.show()
print("Total data points:", df.count())

+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| ID|Reason for absence|age|Education|Son|Social drinker|Social smoker|Pet|Weight|Body mass index|Absenteeism time in hours|Height|
+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| 11|                26| 33|        1|  2|             1|            0|  1|    90|             30|                        4|  1.72|
| 36|                 0| 50|        1|  1|             1|            0|  0|    98|             31|                        0|  1.78|
|  3|                23| 38|        1|  0|             1|            0|  0|    89|             31|                        2|   1.7|
|  7|                 7| 39|        1|  2|             1|            1|  0|    68|             24|                        4|  1.68|
| 11|                23| 33|        1|  2|             1|            0|  1| 

In [15]:
# 'na' stands for Not Available. Using na, we can then use drop. 
# After using show, you'll find that the rows with the null values are gone. 
df.na.drop().show()

# Let's see how many rows of data we have now. 
print("Total data points:", df.count())

+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| ID|Reason for absence|age|Education|Son|Social drinker|Social smoker|Pet|Weight|Body mass index|Absenteeism time in hours|Height|
+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| 11|                26| 33|        1|  2|             1|            0|  1|    90|             30|                        4|  1.72|
| 36|                 0| 50|        1|  1|             1|            0|  0|    98|             31|                        0|  1.78|
|  3|                23| 38|        1|  0|             1|            0|  0|    89|             31|                        2|   1.7|
|  7|                 7| 39|        1|  2|             1|            1|  0|    68|             24|                        4|  1.68|
| 11|                23| 33|        1|  2|             1|            0|  1| 

In [16]:
# It's good practice to specify the column to be filled. 
# In this case, as we know that marital has some missing values, let's fill the null values with the text "UNDISCLOSED".
filled_df = df.na.fill(0)
filled_df.show()
print("Total data points:", df.count())

+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| ID|Reason for absence|age|Education|Son|Social drinker|Social smoker|Pet|Weight|Body mass index|Absenteeism time in hours|Height|
+---+------------------+---+---------+---+--------------+-------------+---+------+---------------+-------------------------+------+
| 11|                26| 33|        1|  2|             1|            0|  1|    90|             30|                        4|  1.72|
| 36|                 0| 50|        1|  1|             1|            0|  0|    98|             31|                        0|  1.78|
|  3|                23| 38|        1|  0|             1|            0|  0|    89|             31|                        2|   1.7|
|  7|                 7| 39|        1|  2|             1|            1|  0|    68|             24|                        4|  1.68|
| 11|                23| 33|        1|  2|             1|            0|  1| 

In [17]:
#linear regression

In [18]:
# Section must be included at the beginning of each new notebook. Remember to change the app name.
# If you're using VirtualBox, change the below to '/home/user/spark-2.1.1-bin-hadoop2.7'
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('linear_regression_adv').getOrCreate()

# If you're getting an error with numpy, please type 'sudo pip3 install numpy --user' into the console.
# If you're getting an error with another package, type 'sudo pip3 install PACKAGENAME --user'. 
# Replace PACKAGENAME with the relevant package (such as pandas, etc).
from pyspark.ml.regression import LinearRegression

# First, let's import the data. Note that we can infer the schema as it's a CSV file.
#df = spark.read.csv("Datasets/boston_housing_data.csv",inferSchema=True,header=True)


In [19]:
# Let's explore. Here's the first row of the data.
print(df.head())

# And the entire data structure. 
df.printSchema()

Row(ID=11, Reason for absence=26, age=33, Education=1, Son=2, Social drinker=1, Social smoker=0, Pet=1, Weight=90, Body mass index=30, Absenteeism time in hours=4, Height=1.72)
root
 |-- ID: integer (nullable = true)
 |-- Reason for absence: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- Son: integer (nullable = true)
 |-- Social drinker: integer (nullable = true)
 |-- Social smoker: integer (nullable = true)
 |-- Pet: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Body mass index: integer (nullable = true)
 |-- Absenteeism time in hours: integer (nullable = true)
 |-- Height: double (nullable = true)



In [20]:
# Now that we understand the data's features, let's use a Python package to neatly describe the data.
import pandas as pd
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
ID,740,18.017567567567568,11.021247263063657,1,36
Reason for absence,740,19.216216216216218,8.43340588279965,0,28
age,740,36.45,6.478772457611868,27,58
Education,740,1.2918918918918918,0.6732380415251598,1,4
Son,740,1.018918918918919,1.0984890195302817,0,4
Social drinker,740,0.5675675675675675,0.49574866720003496,0,1
Social smoker,740,0.07297297297297298,0.2602680502800183,0,1
Pet,740,0.745945945945946,1.3182582913258336,0,8
Weight,740,79.03513513513514,12.883210507177214,56,108


In [21]:
df1=df. select( 'ID','Reason for absence', 'age', 'Son',  'Pet', 'Weight', 'Body mass index','Absenteeism time in hours')
df1.show()

+---+------------------+---+---+---+------+---------------+-------------------------+
| ID|Reason for absence|age|Son|Pet|Weight|Body mass index|Absenteeism time in hours|
+---+------------------+---+---+---+------+---------------+-------------------------+
| 11|                26| 33|  2|  1|    90|             30|                        4|
| 36|                 0| 50|  1|  0|    98|             31|                        0|
|  3|                23| 38|  0|  0|    89|             31|                        2|
|  7|                 7| 39|  2|  0|    68|             24|                        4|
| 11|                23| 33|  2|  1|    90|             30|                        2|
|  3|                23| 38|  0|  0|    89|             31|                        2|
| 10|                22| 28|  1|  4|    80|             27|                        8|
| 20|                23| 36|  4|  0|    65|             23|                        4|
| 14|                19| 34|  2|  0|    95|           

In [22]:
# Now that we understand the data's features, let's use a Python package to neatly describe the data.
import pandas as pd
df1.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
ID,740,18.017567567567568,11.021247263063657,1,36
Reason for absence,740,19.216216216216218,8.43340588279965,0,28
age,740,36.45,6.478772457611868,27,58
Son,740,1.018918918918919,1.0984890195302817,0,4
Pet,740,0.745945945945946,1.3182582913258336,0,8
Weight,740,79.03513513513514,12.883210507177214,56,108
Body mass index,740,26.677027027027027,4.285452223167274,19,38
Absenteeism time in hours,740,6.924324324324324,13.330998100978196,0,120


In [23]:
import pandas as pd

from pandas.plotting import scatter_matrix
numeric_features = [t[0] for t in df1.dtypes if t[1] == 'int' or t[1] == 'double']
sampled_data = df1.select(numeric_features).sample(False, 0.8).toPandas()
axs = pd.plotting.scatter_matrix(sampled_data, figsize=(10, 10))
n = len(sampled_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

In [24]:
# Import VectorAssembler and Vectors
from pyspark.ml.feature import VectorAssembler

# The input columns are the feature column names, and the output column is what you'd like the new column to be named. 
vector_assembler = VectorAssembler(inputCols = [ 'ID','Reason for absence','age','Son','Pet','Weight','Body mass index'],outputCol = 'personal')

# Now that we've created the assembler variable, let's actually transform the data.
vector_output = vector_assembler.transform(df1)

# Using print schema, you see that the features output column has been added. 
vector_output.printSchema()

# You can see that the features column is a DenseVector that combines the various features as expected.
vector_output.head(1)

root
 |-- ID: integer (nullable = true)
 |-- Reason for absence: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- Son: integer (nullable = true)
 |-- Pet: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Body mass index: integer (nullable = true)
 |-- Absenteeism time in hours: integer (nullable = true)
 |-- personal: vector (nullable = true)



[Row(ID=11, Reason for absence=26, age=33, Son=2, Pet=1, Weight=90, Body mass index=30, Absenteeism time in hours=4, personal=DenseVector([11.0, 26.0, 33.0, 2.0, 1.0, 90.0, 30.0]))]

In [25]:
# Because the features have been combined into one vector, we no longer need them. Below we select the features and label.
vector_output = vector_output.filter([ 'personal', 'Absenteesim time in hours'])

# You can see that the dataframe now only contains two columns. 
print(vector_output.head(1))
vector_output.show(3)

TypeError: condition should be string or Column

In [26]:
# Let's do a randomised 70/30 split. Remember, you should explain why you chose a particular split. 
train_data,test_data = vector_output.randomSplit([0.7,0.3])

# Let's see our training data.
train_data.describe().show()

# And our testing data.
test_data.describe().show()

+-------+------------------+-----------------+------------------+------------------+-----------------+------------------+
|summary|Reason for absence|              age|               son|               pet|           Weight|   body mass index|
+-------+------------------+-----------------+------------------+------------------+-----------------+------------------+
|  count|               537|              537|               537|               537|              537|               537|
|   mean|19.202979515828677|  36.487895716946|1.0223463687150838|0.7374301675977654|78.94599627560521| 26.70391061452514|
| stddev| 8.450787655683065|6.451913305565845|1.0975990109143274|1.2943459519548666| 12.7843358191111|4.2386639394973695|
|    min|                 0|               27|                 0|                 0|               56|                19|
|    max|                28|               58|                 4|                 8|              108|                38|
+-------+---------------

In [27]:
#pip install --upgrade patsy
#pip install statsmodels
#import statsmodels.api as sm
#from sklearn import datasets ## imports datasets from scikit-learn
#data = datasets.df() ## loads Boston dataset from datasets library 
#import numpy as np
#import pandas as pd
# define the data/predictors as the pre-set feature names  
#df = pd.DataFrame(data.data, columns=data.feature_names)

# Put the target (housing value -- MEDV) in another DataFrame
#target = pd.DataFrame(data.target, columns=['Absenteeism time in hours'])

In [29]:
# Importing the LR package.
from pyspark.ml.regression import LinearRegression

# Instantiate the instance.
lr = LinearRegression(featuresCol='Absenteesim time in hours', labelCol='ID')

# Fit the training data.
lr_model = lr.fit(train_data)

# Print the coefficients.
print("Coefficients: " + str(lr_model.coefficients))

# Print the intercept.
print("Intercept: " + str(lr_model.intercept) + "\n")

# Summarise the model and print out some evaluation metrics.
training_summary = lr_model.summary

# Print RMSE. 
print("RMSE: " + str(training_summary.rootMeanSquaredError))

# Print R2.
print("R2: " + str(training_summary.r2))

IllegalArgumentException: 'Field "Absenteesim time in hours" does not exist.'